In [14]:
# 基本パラメータ
model_name = "cyberagent/open-calm-3b"   # モデルの名前
peft_name  = "lora-calm-3b"              # 学習用モデル(PEFTモデル)の名前
output_dir = "lora-calm-3b-results"      # 学習結果の出力先

In [15]:
import torch
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer

# モデルの準備
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_8bit=True,
    device_map="auto",
)

# トークンナイザーの準備
tokenizer = AutoTokenizer.from_pretrained(model_name)

# LoRAモデルの準備
model = PeftModel.from_pretrained(
    model, 
    # peft_name, 
    output_dir + "/checkpoint-60",
    device_map="auto"
)

# 評価モード
model.eval()

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): GPTNeoXForCausalLM(
      (gpt_neox): GPTNeoXModel(
        (embed_in): Embedding(52224, 2560)
        (emb_dropout): Dropout(p=0.0, inplace=False)
        (layers): ModuleList(
          (0-31): 32 x GPTNeoXLayer(
            (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
            (post_attention_dropout): Dropout(p=0.0, inplace=False)
            (post_mlp_dropout): Dropout(p=0.0, inplace=False)
            (attention): GPTNeoXAttention(
              (rotary_emb): GPTNeoXRotaryEmbedding()
              (query_key_value): Linear8bitLt(
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2560, out_features=8, bias=False)
                )
         

In [16]:
def generate_query_prompt(data_point):
    return f"""{data_point['text']}

### Response:
"""

In [17]:
# テキスト生成関数の定義
def generate(instruction,input=None,maxTokens=512):
    # 推論
    prompt = generate_query_prompt({'text':instruction})
    input_ids = tokenizer(prompt, return_tensors="pt", truncation=True).input_ids.cuda()
    outputs = model.generate(
        input_ids=input_ids, 
        max_new_tokens=maxTokens, 
        do_sample=True,
        temperature=0.7, 
        top_p=0.75, 
        top_k=40,         
        no_repeat_ngram_size=2,
    )
    outputs = outputs[0].tolist()

    # EOSトークンにヒットしたらデコード完了
    if tokenizer.eos_token_id in outputs:
        eos_index = outputs.index(tokenizer.eos_token_id)
        decoded = tokenizer.decode(outputs[:eos_index])

        # ラベル内容のみ抽出
        sentinel = "### Response:\n"
        sentinelLoc = decoded.find(sentinel)
        if sentinelLoc >= 0:
            response = decoded[sentinelLoc+len(sentinel):]
            # print(response)
        else:
            print('Warning: Expected prompt template to be emitted.  Ignoring output.')
            response = None
    else:
        print('Warning: no <eos> detected ignoring output')
        response = None
        decoded = None

    return response, prompt, decoded

In [18]:
from datasets import Dataset, DatasetDict

dataset_dict = DatasetDict.load_from_disk('dataset_tokenized')

In [19]:
data_set_test = dataset_dict["valid"]

In [20]:
len(data_set_test)

10

In [21]:
# data = dataset_dict["test"][46]
data = data_set_test[2]

response, prompt, decoded = generate(data['text'])
print("---- PROMPT ----")
print(prompt)
print("---- RESPONSE ----")
print(response)
print("---- TARGET ----")
print(data['output'])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


---- PROMPT ----
栃木県（とちぎけん）は、日本の関東地方に位置する県。県庁所在地は宇都宮市。
県北部に日光国立公園があり、観光地・保養地の日光や那須が知られる

### Response:

---- RESPONSE ----
栃木県は県庁所在地の宇都宮市で、県北部日光にある世界遺産の日光の観光地や、保養地である那須があるよ。
---- TARGET ----
栃木県は、日本の関東地方に位置する県で県庁所在地は宇都宮市だよ。
県北部に日光国立公園があり、観光地・保養地の日光や那須が知られてるよ。


In [22]:
results = []
for d in data_set_test:
    response, prompt, decoded = generate(d['text'])
    results.append({
        'text': d['text'],
        'target': d['output'],
        'predicted': response,
    })

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio

In [23]:
results

[{'text': '長崎県（ながさきけん）は、日本の九州地方に位置する県。県庁所在地は長崎市。\n五島列島、壱岐島、対馬など数多くの島嶼を含み、47都道府県中最も島が多いことで知られる。また、多島であるうえにリアス式海岸を多く擁することから海岸線の長さは47都道府県中第2位となっている（※「地形」節にて詳説）',
  'target': '長崎県は、日本の九州地方に位置する県で県庁所在地は長崎市だよ。\n五島列島、壱岐島、対馬など数多くの島嶼を含めて、47都道府県中最も島が多いことで知られていて、また、多島であるうえにリアス式海岸を多く擁することから海岸線の長さは47都道府県中第2位となっているんだ。',
  'predicted': '長崎県は九州にある県で、県庁所在地は長崎市で、島嶼が47都道県中一番多いよ。そして、海に囲まれているから海岸線は47都県の中で一番長く、また多島のうえにリアス海岸をたくさん擁するから海岸線の長さは47番目に長いんだ。'},
 {'text': 'カリブの海賊は、ウォルト・ディズニー自身が設計に携わった最後のアトラクションであり、ウォルトの死から3ヶ月後にディズニーランドで初めて公開された。',
  'target': 'カリブの海賊は、ウォルト・ディズニーが最後に設計したアトラクションで、亡くなってから３か月後にディズニーランドで初公開されたよ。',
  'predicted': 'パイレーツ・オブ・カリビアンは、ディズニーが初めて作ったアトラクションで、ディズニーの死後から3週間後に公開されたんだ。'},
 {'text': '栃木県（とちぎけん）は、日本の関東地方に位置する県。県庁所在地は宇都宮市。\n県北部に日光国立公園があり、観光地・保養地の日光や那須が知られる',
  'target': '栃木県は、日本の関東地方に位置する県で県庁所在地は宇都宮市だよ。\n県北部に日光国立公園があり、観光地・保養地の日光や那須が知られてるよ。',
  'predicted': '栃木県は県庁所在地の宇都宮市で、県の北部に宇都宮公園があり観光地や保養地がたくさんあるよ。'},
 {'text': 'オフラインゲームとしてのウイニングイレブンシリーズとは別に、モードを限定したアイテム課金制の配信ゲーム『myClub（マイクラブ）』(2015

In [24]:
import pandas as pd
df = pd.DataFrame(results)
df.to_excel('valid-results.xlsx')

In [25]:
data_set_train = dataset_dict["train"]

results = []
for d in data_set_train:
    response, prompt, decoded = generate(d['text'])
    results.append({
        'text': d['text'],
        'target': d['output'],
        'predicted': response,
    })

df = pd.DataFrame(results)
df.to_excel('train-results.xlsx')

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
The attentio